# Imaging and deconvolution demonstration

This script makes a fake data set and then deconvolves it. Finally the full and residual visibility are plotted.

In [ ]:
%matplotlib inline

import os
import sys

sys.path.append(os.path.join('..', '..'))

from matplotlib import pylab

pylab.rcParams['figure.figsize'] = (8.0, 8.0)
pylab.rcParams['image.cmap'] = 'rainbow'

import numpy

from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy import constants as const
from astropy.wcs.utils import pixel_to_skycoord

from matplotlib import pyplot as plt

from arl.visibility.operations import create_visibility
from arl.skymodel.operations import create_skycomponent
from arl.image.operations import show_image, export_image_to_fits
from arl.image.deconvolution import deconvolve_cube
from arl.image.iterators import raster_iter
from arl.visibility.iterators import vis_timeslice_iter
from arl.util.testing_support import create_named_configuration, create_test_image
from arl.fourier_transforms.ftprocessor import *

import logging

log = logging.getLogger()
log.setLevel(logging.DEBUG)
log.addHandler(logging.StreamHandler(sys.stdout))

doplot = True

In [ ]:
params = {'npol': 1,
          'spectral_mode': 'channel',
          'channelwidth': 5e7,
          'reffrequency': 1e8}

Construct LOW core configuration

In [ ]:
lowcore = create_named_configuration('LOWBD2')

We create the visibility. This just makes the uvw, time, antenna1, antenna2, weight columns in a table

In [ ]:
times = numpy.arange(- numpy.pi / 4.0, + numpy.pi * 1.001 / 4.0, numpy.pi / 64.0)
frequency = numpy.array([0.8e8, 0.9e8, 1e8, 1.1e8, 1.2e8])

reffrequency = numpy.max(frequency)
phasecentre = SkyCoord(ra=+180.0 * u.deg, dec=-60.0 * u.deg, frame='icrs', equinox=2000.0)
vt = create_visibility(lowcore, times, frequency, weight=1.0, phasecentre=phasecentre, params=params)

Plot the synthesized uv coverage, including for MFS

In [ ]:
plt.clf()
plt.plot(vt.uvw_lambda()[:,0],   vt.uvw_lambda()[:,1], '.', color='b')
plt.plot(-vt.uvw_lambda()[:,0], -vt.uvw_lambda()[:,1], '.', color='r')
plt.xlabel("U (wavelengths)")
plt.ylabel("V (wavelengths)")
plt.show()

Read the venerable test image, constructing an image

In [ ]:
params={}
params['cellsize']=0.00001

m31image = create_test_image(npol=1, nchan=len(frequency), cellsize=params['cellsize'])
nchan, npol, ny, nx = m31image.data.shape
m31image.wcs.wcs.crval[0] = vt.phasecentre.ra.deg
m31image.wcs.wcs.crval[1] = vt.phasecentre.dec.deg
m31image.wcs.wcs.crpix[0] = float(nx // 2)
m31image.wcs.wcs.crpix[1] = float(ny // 2)

fig=show_image(m31image)

In [ ]:
params={}
params['npixel']=4096
params['cellsize']=0.00001
params['timeslice']=1.0
params['nprocessor']=4

vt = predict_timeslice(vt, m31image, params=params)

# To check that we got the prediction right, plot the amplitude of the visibility.
uvdist=numpy.sqrt(vt.data['uvw'][:,0]**2+vt.data['uvw'][:,1]**2)
plt.clf()
plt.plot(uvdist, numpy.abs(vt.data['vis'][:,0,0]), '.')
plt.xlabel('uvdist')
plt.ylabel('Amp Visibility')
plt.show()

Use uniform weighting, and show the density of weights

In [ ]:
params={}
params['npixel']=4096
params['cellsize']=0.00001
params['timeslice']=1.0

model = create_image_from_visibility(vt, params=params)
vt, density, densitygrid = weight_visibility(vt, model)
plt.clf()
plt.imshow(numpy.log(1e-3+densitygrid[0,0,:,:]))
plt.title('UV density')

Make the dirty image and point spread function

In [ ]:
params={}
params['npixel']=4096
params['cellsize']=0.00001
params['timeslice']=1.0
params['nprocessor']=4
dirty, sumwt = invert_timeslice(vt, model, params=params)
psf, sumwt = invert_timeslice(vt, model, dopsf=True, params=params)
dirty.data = dirty.data/sumwt
psf.data = psf.data/sumwt

show_image(dirty)
print("Max, min in dirty image = %.6f, %.6f, sumwt = %f" % (dirty.data.max(), dirty.data.min(), sumwt))

print("Max, min in PSF         = %.6f, %.6f, sumwt = %f" % (psf.data.max(), psf.data.min(), sumwt))

export_image_to_fits(dirty, 'imaging_dirty.fits')
export_image_to_fits(psf, 'imaging_psf.fits')

Deconvolve using clean

In [ ]:
params={'niter':100, 'threshold':0.001, 'fracthresh':0.01, 'scales':[0, 3, 10, 30]}
comp, residual = deconvolve_cube(dirty, psf, params)

# Show the results

fig=show_image(comp)
fig=show_image(residual)

Predict the visibility of the model

In [ ]:
vtmodel = create_visibility(lowcore, times, frequency, weight=1.0, phasecentre=phasecentre)
vtmodel=predict_timeslice(vtmodel, comp, params=params)

Now we will plot the original visibility and the residual visibility.

In [ ]:
uvdist=numpy.sqrt(vt.data['uvw'][:,0]**2+vt.data['uvw'][:,1]**2)
plt.clf()
plt.plot(uvdist, numpy.abs(vt.data['vis'][:,0,0]-vtmodel.data['vis'][:,0,0]), '.', color='r', label='Residual')
plt.plot(uvdist, numpy.abs(vt.data['vis'][:,0,0]), '.', color='b', label='Original')

plt.xlabel('uvdist')
plt.ylabel('Amp Visibility')
plt.legend()
plt.show()